In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [8]:
url = "https://vaksanskrit.com"

## Extraction of Links

### Setting Up the Driver

In [9]:
try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options())
except:
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    print("Running in headless mode.")

In [10]:
driver.get(url)

In [11]:
links = []

There is only one page in the website therefore there is no next button.

### Link Extraction

#### Shortening the search space by specifying the div-class

In [13]:
divs = driver.find_elements(By.CLASS_NAME, "entry-header")
if len(divs) < 1:
    print(f"Some Problem with the link: {url}")

#### Extracting the anchor tags from the divs and then links from anchor tags

In [14]:
for i in range(len(divs)):
    anc = divs[i].find_element(By.TAG_NAME, 'a')
    if(anc==None):
        print(f"Some problem with the link. Continuing...")
        continue
    links.append(anc.get_attribute("href"))

In [17]:
driver.close()

In [18]:
df = pd.DataFrame(links, columns=["url"])

In [19]:
for i in range(len(links)):
    df.loc[len(df)] = links[i]

In [20]:
df.head()

,url
0,https://vaksanskrit.com/?p=4665
1,https://vaksanskrit.com/?p=4663
2,https://vaksanskrit.com/?p=4661
3,https://vaksanskrit.com/?p=4659
4,https://vaksanskrit.com/?p=4657


In [21]:
df.to_csv("./Links/vaksanskrit_links.csv", index=False)

## Extracting the Text Data from the Saved Links

In [23]:
links = pd.read_csv("./Links/vaksanskrit_links.csv")["url"].tolist()

In [25]:
len(links)

200

### Getting the Driver Ready

In [26]:
try:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=Options())
except:
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    print("Running in headless mode.")

### Creating the Folder

In [27]:
import os
if not os.path.exists("./Texts/vaksanskrit"):
    os.makedirs("./Texts/vaksanskrit")

### Creating a File for each Article and Storing the Text in the File

In [32]:
for i in range(len(links)):
    try:
        driver.get(links[i])
    except:
        print(f"Could not open the link: {links[i]}")
    f = os.open(f"./Texts/vaksanskrit/text{i+1}.txt", os.O_WRONLY | os.O_CREAT)
    text = driver.find_element(By.CLASS_NAME, "entry-content").text
    if(len(text)<1):
        print(f"Could not find text in the link: {links[i]}")
        continue
    os.write(f, text.encode())
    os.close(f)

In [33]:
driver.close()